# Discussion 06 Notebook

This notebook is an accompaniment to the associated discussion worksheet handout.

# Section I Window Functions

## Load in the IMDB Performance database

This is a variation of the IMDB database with keys defined. Note that this is a pretty big database! So if you run the below lines, please also remember to delete the `imdb_perf_lecture` afterwards to save space on your limited postgreSQL server.

If you didn't load it in with a previous lecture, load in the `imdb_perf_lecture` database.

In [1]:
!ln -sf ../../proj/proj1/data .
!unzip -u data/imdbdb.zip -d data/

Archive:  data/imdbdb.zip


In [2]:
!psql postgresql://jovyan@127.0.0.1:5432/imdb -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'DROP DATABASE IF EXISTS imdb'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'CREATE DATABASE imdb'
!psql postgresql://jovyan@127.0.0.1:5432/imdb -f data/imdbdb.sql -q

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
 set_config 
------------
 
(1 row)



In [3]:
%reload_ext sql

In [4]:
%sql postgresql://jovyan@127.0.0.1:5432/imdb

Connecting to 'postgresql://jovyan@127.0.0.1:5432/imdb'

## Question 4
In Project 1, we worked with a movie sample table to explore gross earnings for each movie and stored
the results in a view called `movie_gross`. Let’s explore how to leverage **window functions to compute
the average earnings per genre** across movies, as well as the movie's rank within its genre based on gross earnings. Following the view definition, write a `SELECT` query to return the rows for the movie "Mr. & Mrs. Smith" ordered by genre alphabetically.

Create a new view called `movie_avg_genre` with the following columns: `movie_id`, `title`, `gross`, `genre`, `avg_gross_genre`, `genre_rank`.
- `gross`: The gross earnings of the given movie
- `avg_gross_genre`: The average gross earnings of movies with the same genre as the given movie. If a movie belongs to multiple genres, it should appear in multiple rows, with each row corresponding to a different genre. <br/>
- `genre_rank`: The rank of each movies within its genre based on gross earnings (higest gross = 1). If two movies have the same gross earnings, they should share the same rank, and skip the next rank. 

Hint: Check out the movie genre info type, with `info_type_id = 3` from `movie_info_sample`.

In [5]:
%%sql
-- Check out the movie genre info type -- 
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 3
ORDER BY RANDOM()
LIMIT 5;

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

5 rows affected.

id,movie_id,info_type_id,info
6340019,2107997,3,Short
5728685,1706961,3,Family
6371081,2128337,3,Music
6329181,2100610,3,Fantasy
5668519,1667259,3,Music


In [6]:
%%sql
-- Check out the movie genre info type -- 
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 3
ORDER BY RANDOM()
LIMIT 5;

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

5 rows affected.

id,movie_id,info_type_id,info
5827347,1768663,3,Adult
6277049,2067770,3,Drama
5614315,1635227,3,Sci-Fi
6568182,2254225,3,Thriller
5953070,1852406,3,Sci-Fi


In [7]:
%%sql
-- Check out movie_sample -- 
SELECT * 
FROM movie_sample
ORDER BY RANDOM()
LIMIT 5;

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

5 rows affected.

id,title,production_year
2245425,Rites of Spring,2007
2018142,Kickboxer the Champion,1991
1931452,"Gurendaizâ, Gettâ Robo jî, Gurêto Majingâ - Kessen! Daikaijû",1976
2041239,La grande audacia,1913
1670167,Ai teleftaiai imerai tou Odysseos Androutsou,1929


In [8]:
%%sql
-- How we created movie_gross in Project 1, Question 3a --
DROP VIEW IF EXISTS movie_gross CASCADE;
CREATE VIEW movie_gross AS

WITH cleaned as (
    SELECT
        CAST(REGEXP_REPLACE(SUBSTRING(info, '[0-9,]+'), ',', '', 'g') AS float) AS gross,
        movie_id,
        title,
        production_year
    FROM
        movie_info_sample,
        movie_sample
    WHERE
        movie_id = movie_sample.id AND
        info_type_id = 107 AND
        info LIKE '%(USA)%' and info LIKE '$%'
)

SELECT
    max(gross) AS gross,
    movie_id,
    title,
    production_year
FROM
    cleaned
GROUP BY
    movie_id,
    title,
    production_year
ORDER BY gross DESC;

SELECT * FROM movie_gross
LIMIT 5

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

5 rows affected.

gross,movie_id,title,production_year
760507625.0,1704289,Avatar,2009
658672302.0,2438179,Titanic,1997
623357910.0,2346436,The Avengers,2012
534858444.0,2360583,The Dark Knight,2008
460935665.0,2310522,Star Wars,1977


In [ ]:
%%sql
DROP VIEW IF EXISTS movie_avg_genre;
CREATE VIEW movie_avg_genre AS 

-- create a CTE here, which gets the genre(s) and gross of each movie --
WITH movie_genre AS (
    SELECT ____, ____, ____ AS genre, ____
    FROM movie_gross mg
    INNER JOIN  movie_info_sample mis
    ON ____.movie_id = ____.movie_id
    WHERE ____.info_type_id = 3
)

-- for each movie, get its gross and average gross for its genre(s) using window function --
SELECT ____, ____, ____, ____, 
    ____(____) OVER (____) AS ____
    ____() OVER (____) AS ____
FROM movie_genre; 

SELECT * FROM movie_avg_genre
WHERE title = 'Mr. & Mrs. Smith'
ORDER BY genre;

## Question 5
Expanding on the previous question, besides finding the average gross for movies in each genre but **also finding the total gross** for each genre. 
Create a view called `movie_avg_total_genre` with the following columns: `movie_id`, `title`, `gross`, `genre`, `avg_gross`, `total_gross`.
- `gross`: The gross earnings of the given movie
- `avg_gross`: The average gross earnings of movies with the same genre as the given movie. If a movie belongs to multiple genres, it should appear in multiple rows, with each row corresponding to a different genre. <br/>
- `total_gross`: The total gross earning of movies with the same genre as the given movie.
    
**Bonus**: Instead of writing out two separate windows, is it possible to simplify?
    
Documentation: https://www.postgresql.org/docs/current/tutorial-window.html

In [ ]:
%%sql
DROP VIEW IF EXISTS movie_avg_total_genre;
CREATE VIEW movie_avg_total_genre AS 

-- create a CTE here, which gets the genre(s) and gross of each movie --
WITH movie_genre AS (
-- The CTE here is the same as that of Q4 --
    SELECT mg.movie_id, mg.title, mis.info AS genre, mg.gross
    FROM movie_gross mg
    INNER JOIN  movie_info_sample mis
    ON mg.movie_id = mis.movie_id
    WHERE mis.info_type_id = 3
)

-- for each movie, get its gross and average gross for its genre(s) --
SELECT movie_id, title, genre, gross, 
AVG(gross) OVER (PARTITION BY genre) AS avg_gross, -- Same as avg_gross_genre in Q4
____(____) OVER (____) AS ____
FROM movie_genre;

SELECT * FROM movie_avg_total_genre
WHERE title = 'Mr. & Mrs. Smith'
ORDER BY genre;

**Bonus solution** by referencing the documentation here: https://www.postgresql.org/docs/current/tutorial-window.html

In [ ]:
%%sql
DROP VIEW IF EXISTS movie_avg_total_genre;
CREATE VIEW movie_avg_total_genre AS 

-- create a CTE here, which gets the genre(s) and gross of each movie --
WITH movie_genre AS (
-- The CTE here is the same as that of Q4 --
    SELECT mg.movie_id, mg.title, mis.info AS genre, mg.gross
    FROM movie_gross mg
    INNER JOIN  movie_info_sample mis
    ON mg.movie_id = mis.movie_id
    WHERE mis.info_type_id = 3
)

-- for each movie, get its gross and average gross for its genre(s) --
SELECT movie_id, title, genre, gross, 
AVG(gross) OVER w AS avg_gross,
SUM(gross) OVER w AS total_gross
FROM movie_genre
WINDOW w AS (PARTITION BY genre);

SELECT * FROM movie_avg_total_genre
WHERE title = 'Mr. & Mrs. Smith'
ORDER BY genre;

# Section II 
## Data Granularity: Walking a Hierarchy of Arbitrary Depth

## Database Setup

In [1]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS disc07'
!psql -h localhost -c 'CREATE DATABASE disc07'

%reload_ext sql
%sql postgresql://127.0.0.1:5432/disc07

DROP DATABASE
CREATE DATABASE


Connecting to 'postgresql://127.0.0.1:5432/disc07'

In [2]:
!psql postgresql://127.0.0.1:5432/disc07 <disc07.sql

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE EXTENSION
COMMENT
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE MATERIALIZED VIEW
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 6
COPY 176436
COPY 73006
COPY 21
ALTER TABLE
CREATE INDEX
REFRESH MATERIALIZED VIEW


### Initial Exploration

In [3]:
%sql SELECT * FROM nodes ORDER BY tax_id LIMIT 5;

Running query in 'postgresql://127.0.0.1:5432/disc07'

5 rows affected.

tax_id,parent,rank
1,1,no rank
2,131567,superkingdom
6,335928,genus
7,6,species
9,32199,species


In [4]:
%%sql
SELECT tax_id, COUNT(DISTINCT(rank)) FROM nodes GROUP BY tax_id;

Running query in 'postgresql://127.0.0.1:5432/disc07'

73006 rows affected.

tax_id,count
1,1
2,1
6,1
7,1
9,1
10,1
11,1
13,1
14,1
16,1


In [5]:
%sql SELECT * FROM names ORDER BY tax_id LIMIT 5;

Running query in 'postgresql://127.0.0.1:5432/disc07'

5 rows affected.

tax_id,name_txt
1,all
1,root
2,Bacteria
2,bacteria
2,eubacteria


## Question 6
Write a SQL query to find the node representing the `Animalia` kingdom. Your query should return
the kingdom name and its corresponding node information.

In [ ]:
%%sql
-- your code here


## Question 7
Let us drill down into the _Animalia_ kingdom. First, find all children nodes of the Animalia kingdom
(ID 33208). Your query should return the `tax_id` and `rank` of each child node and their parent `Animalia`. Hint: use self-join.

In [ ]:
%%sql
-- your code here
SELECT child.tax_id cid, child.rank crank, parent.tax_id pid, parent.rank prank
FROM ____ AS parent, ____ AS child -- do a self-join here
WHERE parent.____ = child.____ -- only want pairs of nodes that represent a direct parent/child relationship
AND ____; 

## Question 8
Next, find the names of these children nodes of the `Animalia` Kingdom, along with the names of their parents. Your query should
return the `tax_id`, `rank`, and `name_txt` of each child and parent node.

In [ ]:
%config SqlMagic.displaylimit = 10;

In [ ]:
%%sql
-- write a CTE to store the relationship between parent and children --
WITH parent_child AS (
  -- hint: Q7 --
    SELECT child.tax_id cid, child.rank crank, parent.tax_id pid, parent.rank prank
    FROM nodes AS parent, nodes AS child
    WHERE parent.tax_id = child.parent 
)

SELECT pid, prank, parent.name_txt ptext, cid, crank, child.name_txt
FROM ____ AS parent, ____ AS child, parent_child
WHERE parent.____ = parent_child.____
    AND child.____ = parent_child.____
    AND ____
LIMIT 10;

## Question 9
In the biological taxonomy data, it is common for the same rank to have multiple synonym names,
all sharing the same `tax_id`. Write a SQL query to return the parent node (Animalia) and its direct
child nodes. For each phylum (child node), aggregate all its synonym names (for each group of parent names and for each group of children names) into a single JSON array, using the `json_agg(column)` function.

In [ ]:
%%sql
-- your code here
WITH parent_child AS (
 -- finish your CTE here --
    SELECT child.tax_id cid, child.rank crank, parent.tax_id pid, parent.rank prank
    FROM nodes AS parent, nodes AS child
    WHERE parent.tax_id = child.parent 
)

SELECT pid, prank, json_agg(____), cid, crank, json_agg(____)
FROM ____, ____, ____
WHERE 
    ____
    AND ____
GROUP BY ____, ____, ____, ____;

## **Optional, Challenge:** Question 10  
How can we drill down one more layer? What if we want to get the names of all the classes under the Animalia kingdom?

In [9]:
%%sql

WITH gparent_parent_child AS (
    SELECT grandparent.tax_id gid, grandparent.rank grank, child.tax_id cid, child.rank crank, parent.tax_id pid, parent.rank prank
    FROM nodes AS grandparent, nodes AS parent, nodes AS child
    WHERE grandparent.tax_id = parent.parent
    AND parent.tax_id = child.parent 
)

SELECT gid, grank, json_agg(grandparent.name_txt) AS gnames, 
       pid, prank, json_agg(parent.name_txt) AS pnames, 
       cid, crank, json_agg(child.name_txt) AS cnames
FROM names grandparent, names parent, names child, gparent_parent_child gpc
WHERE grandparent.tax_id = gpc.gid
    AND parent.tax_id = gpc.pid
    AND child.tax_id = gpc.cid
    AND grandparent.tax_id = 33208
GROUP BY gid, grank, pid, prank, cid, crank;

Running query in 'postgresql://127.0.0.1:5432/disc07'

8 rows affected.

gid,grank,gnames,pid,prank,pnames,cid,crank,cnames
33208,kingdom,"['Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia']",6040,phylum,"['Porifera', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'Parazoa', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges']",6042,class,"['Demospongiae', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Ceractinomorpha', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Monaxonida', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetractinomorpha', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Tetraxonida', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Ceractinomorpha', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Demospongiae', 'Monaxonida', '